# Double-Bracket Iteration Strategy: Pauli-Z products

In this example, we demonstrate the usage of a DBI strategy, where the diagonal operators for double bracket iterations are variationally chosen from all possible local Pauli-Z operators.

### Initial setup

In [ ]:
!python -m pip install seaborn # plotting library
!python -m pip install hyperopt # required to optimize the DBF step

In [ ]:
from copy import copy, deepcopy

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from qibo import hamiltonians, set_backend
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketIteration
from qibo.models.dbi.utils import *

Below are some useful functions to visualize the diagonalization process.

In [ ]:
def visualize_matrix(matrix, title=""):
    """Visualize absolute values of a matrix in a heatmap form."""
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_title(title)
    try:
        im = ax.imshow(np.absolute(matrix), cmap="inferno")
    except TypeError:
        im = ax.imshow(np.absolute(matrix.get()), cmap="inferno")
    fig.colorbar(im, ax=ax)


def visualize_drift(h0, h):
    """Visualize drift of the evolved hamiltonian w.r.t. h0."""
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_title(r"Drift: $|\hat{H}_0 - \hat{H}_{1}|$")
    try:
        im = ax.imshow(np.absolute(h0 - h), cmap="inferno")
    except TypeError:
        im = ax.imshow(np.absolute((h0 - h).get()), cmap="inferno")

    fig.colorbar(im, ax=ax)


def plot_histories(loss_histories: list, steps: list, labels: list = None):
    """Plot off-diagonal norm histories over a sequential evolution."""
    plt.figure(figsize=(5, 5 * 6 / 8))
    if len(steps) == 1:
        # fixed_step
        x_axis = [i * steps[0] for i in range(len(loss_histories))]
    else:
        x_axis = [sum(steps[:k]) for k in range(1, len(steps) + 1)]
    plt.plot(x_axis, loss_histories, "-o")

    x_labels_rounded = [round(x, 2) for x in x_axis]
    x_labels_rounded = [0] + x_labels_rounded[0:5] + [max(x_labels_rounded)]
    x_labels_rounded.pop(3)
    plt.xticks(x_labels_rounded)

    y_labels_rounded = [round(y, 1) for y in loss_histories]
    y_labels_rounded = y_labels_rounded[0:5] + [min(y_labels_rounded)]
    plt.yticks(y_labels_rounded)

    if labels is not None:
        labels_copy = copy(labels)
        labels_copy.insert(0, "Initial")
        for i, label in enumerate(labels_copy):
            plt.text(x_axis[i], loss_histories[i], label)

    plt.grid()
    plt.xlabel(r"Flow duration $s$")
    plt.title("Loss function histories")

# Example: TFIM

As an example, we consider the Transverse Field Ising Model (TFIM):
$$ H_{\rm TFIM} = - \sum_{i=1}^{N}\bigl( Z_i Z_{i+1} + h X_i \bigr),$$
which is already implemented in `Qibo`. For this tutorial we set $N=5$ and $h=3$.

In [ ]:
# set the qibo backend (we suggest qibojit if N >= 20)
# alternatives: tensorflow (not optimized), numpy (when CPU not supported by jit)
set_backend("qibojit", "numba")

# hamiltonian parameters
nqubits = 5
h = 3

# define the hamiltonian
H_TFIM = hamiltonians.TFIM(nqubits=nqubits, h=h)

# initialize class
# Note: use deepcopy to prevent h being edited
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator)
print("Initial off diagonal norm", dbi.off_diagonal_norm)

### Generate local Pauli-Z operators

In [ ]:
generate_local_Z = generate_Z_operators(nqubits)
Z_ops = list(generate_local_Z.values())
Z_names = list(generate_local_Z.keys())

## Iteration from a list of operators
The idea of this strategy is to chose the Z operator that reduces the off-diagonal norm of the hamiltonian most efficiently. Given a list of operators (np.array), the function `select_best_dbr_generator_and_run` searches for the maximum decrease in off-diagonal norm for each operator and runs one double bracket rotation using the optimal operator from the list.

In [ ]:
NSTEPS = 10
Z_optimal = []
# add in initial values for plotting
off_diagonal_norm_history = [dbi.off_diagonal_norm]
steps = [0]
for _ in range(NSTEPS):
    idx, step = select_best_dbr_generator_and_run(dbi, Z_ops, compare_canonical=False)
    off_diagonal_norm_history.append(dbi.off_diagonal_norm)
    steps.append(steps[-1]+step)
    Z_optimal.append(Z_names[idx])
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {step} with operator {Z_optimal[-1]}")

In [ ]:
plot_histories(off_diagonal_norm_history, steps, Z_optimal)

It is worth noting that due to the nature of `hyperopt`, the iterations may be unstable and multiple runs may be required for the optimal result (alternatively, we can perform a grid search on the optimal step). Hence, it is sometimes needed to adjust its parameters including the following:

- step_min
- step_max
- max_evals

### Compare with canonical

We compare the effectiveness at diagonalzation between the Pauli-Z operators and the canonical generator:

$$ d = [H,\sigma(H)]$$

In [ ]:
# set the qibo backend (we suggest qibojit if N >= 20)
# alternatives: tensorflow (not optimized), numpy (when CPU not supported by jit)
set_backend("qibojit", "numba")


# initialize class|
# Note: use deepcopy to prevent h being edited
dbi_canonical = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.canonical)
print("Initial off diagonal norm", dbi_canonical.off_diagonal_norm)

In [ ]:
off_diagonal_norm_history_canonical = [dbi_canonical.off_diagonal_norm]
steps_canonical = [0]
steps_canonical_plot = [0]
for s in range(NSTEPS):
    # same settings as iteration from list
    step = dbi_canonical.hyperopt_step(
        step_min = 1e-5,
        step_max = 1,
        space = hp.uniform,
        optimizer = tpe,
        max_evals = 100,
    )
    print(f"New optimized step at iteration {s+1}/{NSTEPS}: {step}")
    dbi_canonical(step=step)
    off_diagonal_norm_history_canonical.append(dbi_canonical.off_diagonal_norm)
    steps_canonical.append(step)
    steps_canonical_plot.append(steps_canonical_plot[-1]+step)

In [ ]:
plt.figure()
# plt.plot(steps, off_diagonal_norm_history, label="Pauli-Z")
# plt.plot(steps_canonical, off_diagonal_norm_history_canonical, label="Canonical")
plt.plot(off_diagonal_norm_history, label="Pauli-Z")
plt.plot(off_diagonal_norm_history_canonical, label="Canonical")
plt.xlabel("Iterations")
plt.ylabel("Norm off-diagonal restriction")
plt.title("Compare Variational Pauli-Z with Canonical")
plt.legend()

Here, we make 2 observations:

1. The canonical strategy has a steeper decrease at the beginning than Pauli-Z operators.
2. However, the canonical strategy is also prone to getting stuck at a local minimum and hence resultting in a lesser degree of diagonalization.

Therefore, we explore the possibility of mixing the two strategies by including the canonical generator in the list.

## Mixed strategy: optimal at each step

In [ ]:
# set the qibo backend (we suggest qibojit if N >= 20)
# alternatives: tensorflow (not optimized), numpy (when CPU not supported by jit)
set_backend("qibojit", "numba")

# hamiltonian parameters
nqubits = 5
h = 3

# define the hamiltonian
H_TFIM = hamiltonians.TFIM(nqubits=nqubits, h=h)

# initialize class
# Note: use deepcopy to prevent h being edited
dbi_mixed = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator)
print("Initial off diagonal norm", dbi.off_diagonal_norm)

In [ ]:
NSTEPS = 10
Z_optimal_mixed = []
# add in initial values for plotting
off_diagonal_norm_history_mixed = [dbi_mixed.off_diagonal_norm]
steps = [0]
for _ in range(NSTEPS):
    idx, step = select_best_dbr_generator_and_run(dbi_mixed, Z_ops, compare_canonical=True)
    off_diagonal_norm_history_mixed.append(dbi_mixed.off_diagonal_norm)
    steps.append(steps[-1]+step)
    if idx == len(Z_names):
        Z_optimal_mixed.append('Canonical')
    else:
        Z_optimal_mixed.append(Z_names[idx])
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {step} with operator {Z_optimal_mixed[-1]}")

In [ ]:
plt.figure()
# plt.plot(steps, off_diagonal_norm_history, label="Pauli-Z")
# plt.plot(steps_canonical, off_diagonal_norm_history_canonical, label="Canonical")
plt.plot(off_diagonal_norm_history, label="Pauli-Z")
plt.plot(off_diagonal_norm_history_canonical, label="Canonical")
plt.plot(off_diagonal_norm_history_mixed, label="Mixed")
plt.xlabel("Iterations")
plt.ylabel("Norm off-diagonal restriction")
plt.title("Compare Variational Pauli-Z with Canonical")
plt.legend()

After a few tests, we realize that the mixed strategy does not always outperform just using Pauli-Z operators. This could be caused by 2 reasons: 

1. Unstability of hyperopt
2. Tendency of canonical operator to get stuck

## Mixed strategy: initial canonical

Since the canonical double bracket iteration performs better at the initial steps, we attempt to combine the two strategies: iterate a few steps using the canonical bracket before switching to the variational Z-operators.

In [ ]:
dbi_mixed_can= DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.canonical)
print("Initial off diagonal norm", dbi_mixed_can.off_diagonal_norm)

In [ ]:
print(steps_canonical)
print(off_diagonal_norm_history_canonical)

In [ ]:
# Run the initial iterations using canonical iterations
off_diagonal_norm_history_mixed_can = [dbi_mixed_can.off_diagonal_norm]
steps_mixed_can = [0]
cannonical_NSTEPS = 2
for i in range(cannonical_NSTEPS):
    step = steps_canonical[i+1]
    dbi_mixed_can(step=step)
    off_diagonal_norm_history_mixed_can.append(dbi_mixed_can.off_diagonal_norm)
    steps_mixed_can.append(step)
    
print("After 2 steps, off diagonal norm:", dbi_mixed_can.off_diagonal_norm)
print("By comparison, the Pauli-Z:", off_diagonal_norm_history[2])

In [ ]:
# Continue the remaining steps with Pauli-Z operators
Z_optimal_mixed_can = ["Cannonical" for _ in range(cannonical_NSTEPS)]
remaining_NSTEPS = NSTEPS - cannonical_NSTEPS
dbi_mixed_can.mode = DoubleBracketGeneratorType.single_commutator
for _ in range(remaining_NSTEPS):
    idx, step = select_best_dbr_generator_and_run(dbi_mixed_can, Z_ops, compare_canonical=False)
    off_diagonal_norm_history_mixed_can.append(dbi_mixed_can.off_diagonal_norm)
    steps_mixed_can.append(step)
    if idx == len(Z_ops):
        Z_optimal_mixed_can.append("canonical")
    else:
        Z_optimal_mixed_can.append(Z_names[idx])
    print(f"New optimized step at iteration {_+1}/{remaining_NSTEPS}: {step} with operator {Z_optimal_mixed_can[-1]}")

In [ ]:
print(off_diagonal_norm_history_mixed_can)
print(steps_mixed_can)
print(Z_optimal_mixed_can)

In [ ]:
plot_histories(off_diagonal_norm_history_mixed_can, steps_mixed_can, Z_optimal_mixed_can)

We see that after two canonical steps, the diagonalization gets stuck at the same local minimum as the canonical trial. Hence, it may not be ideal to run the canonical generator at initial steps.